<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Matrículas---Censo-Escolar" data-toc-modified-id="Matrículas---Censo-Escolar-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Matrículas - Censo Escolar</a></span><ul class="toc-item"><li><span><a href="#Dados-faltantes" data-toc-modified-id="Dados-faltantes-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Dados faltantes</a></span></li><li><span><a href="#Variável-de-evasão" data-toc-modified-id="Variável-de-evasão-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Variável de evasão</a></span></li><li><span><a href="#Variáveis-de-caracterização-dos-alunos" data-toc-modified-id="Variáveis-de-caracterização-dos-alunos-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Variáveis de caracterização dos alunos</a></span><ul class="toc-item"><li><span><a href="#Pessoal" data-toc-modified-id="Pessoal-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Pessoal</a></span></li><li><span><a href="#Transporte" data-toc-modified-id="Transporte-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Transporte</a></span></li><li><span><a href="#Localização" data-toc-modified-id="Localização-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Localização</a></span></li><li><span><a href="#Dados-da-turma" data-toc-modified-id="Dados-da-turma-1.3.4"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>Dados da turma</a></span></li></ul></li><li><span><a href="#Filtragem-das-variáveis" data-toc-modified-id="Filtragem-das-variáveis-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Filtragem das variáveis</a></span></li></ul></li><li><span><a href="#Infos-das-escolas:-Censo-2017" data-toc-modified-id="Infos-das-escolas:-Censo-2017-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Infos das escolas: Censo 2017</a></span><ul class="toc-item"><li><span><a href="#Checando-distribuições" data-toc-modified-id="Checando-distribuições-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Checando distribuições</a></span></li></ul></li><li><span><a href="#Nível-socioeconômico" data-toc-modified-id="Nível-socioeconômico-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Nível socioeconômico</a></span><ul class="toc-item"><li><span><a href="#Finalizando-a-base" data-toc-modified-id="Finalizando-a-base-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Finalizando a base</a></span></li></ul></li></ul></div>

# Tratamenfo inicial das bases
----

- Dados faltantes e filtragem
- Criação de variáveis
- Junção das bases

In [2]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

import os
from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import pandas_profiling

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

## Matrículas - Censo Escolar

In [3]:
os.listdir(TREAT_PATH / 'censo')

['cleaned_escolas_rj_2017.csv',
 'cleaned_escolas_rj_2016.csv',
 'matriculas_rj_2017duplicado.csv',
 'cleaned_matriculas_rj_2016.csv',
 'cleaned_matriculas_rj_2017.csv',
 'matriculas_rj_2016duplicado.csv']

In [4]:
alunos_rj_16 = pd.read_csv(TREAT_PATH / 'censo' / 'cleaned_matriculas_rj_2016.csv')
len(alunos_rj_16)

570111

In [5]:
alunos_rj_16 = alunos_rj_16[alunos_rj_16['TP_ETAPA_ENSINO'].isin([25, 31])] 
alunos_rj_16 = alunos_rj_16[alunos_rj_16['TP_DEPENDENCIA'] == 2] # Escolas estaduais

alunos_rj_16.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182838 entries, 118 to 552067
Data columns (total 93 columns):
Unnamed: 0                       182838 non-null int64
NU_ANO_CENSO                     182838 non-null int64
ID_MATRICULA                     182838 non-null int64
NU_DIA                           182838 non-null int64
NU_MES                           182838 non-null int64
NU_ANO                           182838 non-null int64
NU_IDADE_REFERENCIA              182838 non-null int64
NU_IDADE                         182838 non-null int64
NU_DURACAO_TURMA                 182838 non-null int64
NU_DUR_ATIV_COMP_MESMA_REDE      182838 non-null int64
NU_DUR_ATIV_COMP_OUTRAS_REDES    182838 non-null int64
NU_DUR_AEE_MESMA_REDE            182838 non-null int64
NU_DUR_AEE_OUTRAS_REDES          182838 non-null int64
NU_DIAS_ATIVIDADE                182838 non-null int64
TP_SEXO                          182838 non-null int64
TP_COR_RACA                      182838 non-null int64
TP_NACI

In [6]:
alunos_rj_17 = pd.read_csv(TREAT_PATH / 'censo' / 'cleaned_matriculas_rj_2017.csv')
len(alunos_rj_17)

551630

* Para ter o percentual mais aproximado e quem evadiu, precisamos separar alunos que permaneceram no ensino médio no ano seguinte (na mesma escola, ou em outra estadual ou particular)

In [7]:
alunos_rj_17 = alunos_rj_17[alunos_rj_17['TP_ETAPA_ENSINO'].isin([25, 26, 31, 32])] # 1a série do MED
# alunos_rj_17 = alunos_rj_17[alunos_rj_17['TP_DEPENDENCIA'] == 2] # Escolas estaduais

alunos_rj_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 394488 entries, 0 to 545157
Data columns (total 93 columns):
Unnamed: 0                       394488 non-null int64
NU_ANO_CENSO                     394488 non-null int64
ID_MATRICULA                     394488 non-null int64
NU_DIA                           394488 non-null int64
NU_MES                           394488 non-null int64
NU_ANO                           394488 non-null int64
NU_IDADE_REFERENCIA              394488 non-null int64
NU_IDADE                         394488 non-null int64
NU_DURACAO_TURMA                 394488 non-null int64
NU_DUR_ATIV_COMP_MESMA_REDE      394488 non-null int64
NU_DUR_ATIV_COMP_OUTRAS_REDES    394488 non-null int64
NU_DUR_AEE_MESMA_REDE            394488 non-null int64
NU_DUR_AEE_OUTRAS_REDES          394488 non-null int64
NU_DIAS_ATIVIDADE                394488 non-null int64
TP_SEXO                          394488 non-null int64
TP_COR_RACA                      394488 non-null int64
TP_NACION

### Dados faltantes

Não entram (< 30%):
- 35 variáveis têm menos que 30% de dado
- Variáveis de endereço e especificações de deficiência / superdotação (ex: cegueira, surdez, etc)
- Variáveis irrelevaantes: específicas de escolas privadas

< 95%:
- 12 dentre as 58 restantes
- Todas especificações de transporte (responsável e tipo específico de transporte)

In [8]:
n = alunos_rj_16.count()/182838 > 0.3
n.value_counts()

True     58
False    35
dtype: int64

In [9]:
n[n == False].index

Index(['CO_UF_END', 'CO_MUNICIPIO_END', 'IN_CEGUEIRA', 'IN_BAIXA_VISAO',
       'IN_SURDEZ', 'IN_DEF_AUDITIVA', 'IN_SURDOCEGUEIRA', 'IN_DEF_FISICA',
       'IN_DEF_INTELECTUAL', 'IN_DEF_MULTIPLA', 'IN_AUTISMO',
       'IN_SINDROME_ASPERGER', 'IN_SINDROME_RETT', 'IN_TRANSTORNO_DI',
       'IN_SUPERDOTACAO', 'IN_RECURSO_LEDOR', 'IN_RECURSO_TRANSCRICAO',
       'IN_RECURSO_INTERPRETE', 'IN_RECURSO_LIBRAS', 'IN_RECURSO_LABIAL',
       'IN_RECURSO_BRAILLE', 'IN_RECURSO_AMPLIADA_16',
       'IN_RECURSO_AMPLIADA_20', 'IN_RECURSO_AMPLIADA_24', 'IN_RECURSO_NENHUM',
       'TP_INGRESSO_FEDERAIS', 'CO_CURSO_EDUC_PROFISSIONAL',
       'TP_CATEGORIA_ESCOLA_PRIVADA', 'IN_CONVENIADA_PP',
       'TP_CONVENIO_PODER_PUBLICO', 'IN_MANT_ESCOLA_PRIVADA_EMP',
       'IN_MANT_ESCOLA_PRIVADA_ONG', 'IN_MANT_ESCOLA_PRIVADA_SIND',
       'IN_MANT_ESCOLA_PRIVADA_SIST_S', 'IN_MANT_ESCOLA_PRIVADA_S_FINS'],
      dtype='object')

In [10]:
alunos_rj_16 = alunos_rj_16[n[n == True].index]

In [11]:
n = alunos_rj_16.count()/182838 > 0.95
n.value_counts()

True     46
False    12
dtype: int64

In [12]:
alunos_rj_16[n[n == False].index].count()/182838

TP_RESPONSAVEL_TRANSPORTE    0.66705
IN_TRANSP_VANS_KOMBI         0.66705
IN_TRANSP_MICRO_ONIBUS       0.66705
IN_TRANSP_ONIBUS             0.66705
IN_TRANSP_BICICLETA          0.66705
IN_TRANSP_TR_ANIMAL          0.66705
IN_TRANSP_OUTRO_VEICULO      0.66705
IN_TRANSP_EMBAR_ATE5         0.66705
IN_TRANSP_EMBAR_5A15         0.66705
IN_TRANSP_EMBAR_15A35        0.66705
IN_TRANSP_EMBAR_35           0.66705
IN_TRANSP_TREM_METRO         0.66705
dtype: float64

### Variável de evasão

In [13]:
# Total de alunos 1o e 2o MED em 2016
n = len(alunos_rj_16['ID'])
n

182838

In [14]:
df_permanencia_16_17 = alunos_rj_16[alunos_rj_16['ID'].isin(alunos_rj_17['ID'])]['ID']
len(df_permanencia_16_17)

120156

In [15]:
# Percentual de permanência
len(df_permanencia_16_17)/n

0.6571719226856562

In [16]:
alunos_rj_16['IN_EVASAO'] = np.where(alunos_rj_16['ID'].isin(df_permanencia_16_17), 0, 1)
alunos_rj_16.head()

,Unnamed: 0,NU_ANO_CENSO,ID_MATRICULA,NU_DIA,NU_MES,NU_ANO,NU_IDADE_REFERENCIA,NU_IDADE,NU_DURACAO_TURMA,NU_DUR_ATIV_COMP_MESMA_REDE,NU_DUR_ATIV_COMP_OUTRAS_REDES,NU_DUR_AEE_MESMA_REDE,NU_DUR_AEE_OUTRAS_REDES,NU_DIAS_ATIVIDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,CO_PAIS_ORIGEM,CO_UF_NASC,CO_MUNICIPIO_NASC,TP_ZONA_RESIDENCIAL,TP_OUTRO_LOCAL_AULA,IN_TRANSPORTE_PUBLICO,TP_RESPONSAVEL_TRANSPORTE,IN_TRANSP_VANS_KOMBI,IN_TRANSP_MICRO_ONIBUS,IN_TRANSP_ONIBUS,IN_TRANSP_BICICLETA,IN_TRANSP_TR_ANIMAL,IN_TRANSP_OUTRO_VEICULO,IN_TRANSP_EMBAR_ATE5,IN_TRANSP_EMBAR_5A15,IN_TRANSP_EMBAR_15A35,IN_TRANSP_EMBAR_35,IN_TRANSP_TREM_METRO,IN_NECESSIDADE_ESPECIAL,TP_MEDIACAO_DIDATICO_PEDAGO,IN_ESPECIAL_EXCLUSIVA,IN_REGULAR,IN_EJA,IN_PROFISSIONALIZANTE,TP_ETAPA_ENSINO,ID_TURMA,TP_UNIFICADA,TP_TIPO_TURMA,CO_ENTIDADE,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_REGULAMENTACAO,TP_LOCALIZACAO_DIFERENCIADA,IN_EDUCACAO_INDIGENA,ID,IN_EVASAO
118,118,2016,162295208,2,12,1999,16,17,680,0,0,0,0,5,2,0,1,76,33.0,3301009.0,1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,1,0,1,31,6062013,0,0,33009627,3,3302,33003,33,3301009,330100905,2,1,1,0,0,112132289130,0
122,122,2016,114897083,3,12,1995,20,21,680,0,0,0,0,5,2,0,1,76,33.0,3301009.0,1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,1,0,1,31,6062013,0,0,33009627,3,3302,33003,33,3301009,330100905,2,1,1,0,0,113220290328,1
124,124,2016,189344,7,5,2000,16,16,680,0,0,0,0,5,2,0,1,76,33.0,3301009.0,1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,1,0,1,31,6060199,0,0,33009627,3,3302,33003,33,3301009,330100905,2,1,1,0,0,112752673352,0
142,142,2016,239895,19,8,1999,16,17,680,0,0,0,0,5,2,1,1,76,33.0,3304557.0,1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,1,0,1,31,6070919,0,0,33009627,3,3302,33003,33,3301009,330100905,2,1,1,0,0,117459545545,0
150,150,2016,185051,11,10,1998,17,18,680,0,0,0,0,5,2,0,1,76,33.0,3304557.0,1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,1,0,1,31,6060199,0,0,33009627,3,3302,33003,33,3301009,330100905,2,1,1,0,0,113868062938,1


In [17]:
t = alunos_rj_16[alunos_rj_16['ID'].isin(alunos_rj_17['ID'])]
t['ID'].values[:5]

array([112132289130, 112752673352, 117459545545, 110337630479,
       112508075220])

### Variáveis de caracterização dos alunos

#### Pessoal

* Raça: maioria (46%) não declarada

In [18]:
alunos_rj_16['TP_COR_RACA'].value_counts() # Maioria não declarada

0    84623
3    51340
1    31478
2    14930
4      356
5      111
Name: TP_COR_RACA, dtype: int64

In [20]:
alunos_rj_16['TP_COR_RACA'].value_counts(normalize=True)

0    0.462830
3    0.280795
1    0.172163
2    0.081657
4    0.001947
5    0.000607
Name: TP_COR_RACA, dtype: float64

In [21]:
# Percentual de preenchimento
n = len(alunos_rj_16[alunos_rj_16['TP_COR_RACA'].isnull() == False])
n/len(alunos_rj_16)

1.0

* Sexo: ok

In [22]:
# Percentual de preenchimento
n = len(alunos_rj_16[alunos_rj_16['TP_SEXO'].isnull() == False])
n/len(alunos_rj_16)

1.0

In [23]:
n = alunos_rj_16['TP_SEXO'].value_counts() # Maioria homem
n

1    94256
2    88582
Name: TP_SEXO, dtype: int64

In [24]:
alunos_rj_16['TP_SEXO'].value_counts(normalize=True)

1    0.515516
2    0.484484
Name: TP_SEXO, dtype: float64

In [25]:
# Evadidos: percentual semelhante, pouco mais homens
ev = alunos_rj_16[alunos_rj_16['IN_EVASAO'] == 1]['TP_SEXO'].value_counts()
ev

1    34306
2    28376
Name: TP_SEXO, dtype: int64

In [26]:
ev/n

1    0.363966
2    0.320336
Name: TP_SEXO, dtype: float64

* Idade: ok

In [27]:
# Percentual de preenchimento
n = len(alunos_rj_16[alunos_rj_16['NU_IDADE_REFERENCIA'].isnull() == False])
n/len(alunos_rj_16)

1.0

In [28]:
alunos_rj_16['NU_IDADE_REFERENCIA'].value_counts()[:10] # Maioria entre 14-17, também idades muito discrepantes

15    61605
16    54432
17    35777
18    14741
14     7120
19     4763
20     1959
21      731
22      376
23      219
Name: NU_IDADE_REFERENCIA, dtype: int64

In [29]:
# Distorção idade série: Maior ou igual a 17 anos para o 10 ano do MED
bins_idade = pd.cut(alunos_rj_16['NU_IDADE_REFERENCIA'], [-np.inf, 14, 16, 18, 20, np.inf])
bins_idade.value_counts(normalize=True).sort_index()

(-inf, 14.0]    0.039669
(14.0, 16.0]    0.634644
(16.0, 18.0]    0.276299
(18.0, 20.0]    0.036765
(20.0, inf]     0.012623
Name: NU_IDADE_REFERENCIA, dtype: float64

In [30]:
alunos_rj_16['IN_DISTORCAO'] = np.where(alunos_rj_16['NU_IDADE_REFERENCIA'] >= 17, 1, 0)

In [31]:
n = alunos_rj_16['IN_DISTORCAO'].value_counts()
n

0    123290
1     59548
Name: IN_DISTORCAO, dtype: int64

In [32]:
alunos_rj_16['IN_DISTORCAO'].value_counts(normalize=True)

0    0.674313
1    0.325687
Name: IN_DISTORCAO, dtype: float64

In [33]:
# Evadidos: maior percentual de alunos com distorção idade-série
ev = alunos_rj_16[alunos_rj_16['IN_EVASAO'] == 1]['IN_DISTORCAO'].value_counts()
ev

1    34554
0    28128
Name: IN_DISTORCAO, dtype: int64

In [34]:
ev/n

0    0.228145
1    0.580271
Name: IN_DISTORCAO, dtype: float64

* Deficiência: 99% não tem

In [35]:
alunos_rj_16['IN_NECESSIDADE_ESPECIAL'].value_counts(normalize=True) # Quase ninguém com necessidade especial

0    0.994728
1    0.005272
Name: IN_NECESSIDADE_ESPECIAL, dtype: float64

In [36]:
# Percentual de preenchimento
n = len(alunos_rj_16[alunos_rj_16['IN_NECESSIDADE_ESPECIAL'].isnull() == False])
n/len(alunos_rj_16)

1.0

* Nacionalidade: 99% brasil

In [37]:
alunos_rj_16['TP_NACIONALIDADE'].value_counts(normalize=True)

1    0.999448
3    0.000509
2    0.000044
Name: TP_NACIONALIDADE, dtype: float64

* Local de nascimento: 94% no RJ

In [38]:
alunos_rj_16['CO_UF_NASC'].value_counts(normalize=True)

33.0    0.940237
31.0    0.010272
35.0    0.007628
29.0    0.007234
25.0    0.006851
23.0    0.005472
32.0    0.004115
26.0    0.003727
21.0    0.003705
15.0    0.001773
27.0    0.001445
22.0    0.001275
24.0    0.001177
53.0    0.000897
28.0    0.000821
41.0    0.000679
43.0    0.000509
13.0    0.000487
52.0    0.000405
50.0    0.000334
51.0    0.000246
42.0    0.000197
11.0    0.000186
17.0    0.000142
14.0    0.000077
12.0    0.000077
16.0    0.000033
Name: CO_UF_NASC, dtype: float64

In [39]:
alunos_rj_16['CO_MUNICIPIO_NASC'].value_counts(normalize=True)

3304557.0    0.427297
3301702.0    0.046230
3304904.0    0.043642
3303500.0    0.034054
3305109.0    0.031154
3301009.0    0.028659
3300456.0    0.028401
3303302.0    0.026831
3303906.0    0.016740
3301900.0    0.016691
3306305.0    0.016072
3305802.0    0.013123
3302502.0    0.012630
3303203.0    0.012143
3302403.0    0.010578
3300100.0    0.010058
3300407.0    0.009434
3304144.0    0.009210
3303401.0    0.009035
3303609.0    0.008914
3300704.0    0.008438
3304201.0    0.008033
3302270.0    0.006704
3300209.0    0.006145
3302205.0    0.005987
3304300.0    0.005549
3302858.0    0.005505
3306008.0    0.005264
3300308.0    0.005002
3302007.0    0.004974
               ...   
2703007.0    0.000005
2703908.0    0.000005
2303808.0    0.000005
2704708.0    0.000005
2303501.0    0.000005
2705200.0    0.000005
2616183.0    0.000005
2305704.0    0.000005
2615102.0    0.000005
2306108.0    0.000005
2309805.0    0.000005
2309706.0    0.000005
2309607.0    0.000005
2309508.0    0.000005
2309409.0 

In [40]:
v = alunos_rj_16['CO_MUNICIPIO_NASC'] != alunos_rj_16['CO_MUNICIPIO']
v.value_counts(normalize=True)

False    0.677069
True     0.322931
dtype: float64

* Residência: 95% urbano, não tem cidade (< 7% preenchido)

In [41]:
alunos_rj_16['TP_ZONA_RESIDENCIAL'].value_counts(normalize=True)

1    0.956218
2    0.043782
Name: TP_ZONA_RESIDENCIAL, dtype: float64

#### Transporte

Tudo preenchido!! 2/3 utiliza transporte público

In [42]:
# Percentual de preenchimento
n = len(alunos_rj_16[alunos_rj_16['IN_TRANSPORTE_PUBLICO'].isnull() == False])
n/len(alunos_rj_16)

1.0

In [43]:
n = alunos_rj_16['IN_TRANSPORTE_PUBLICO'].value_counts() # Já tem uma variação mais razoável: 2/3 utiliza
n

1    121962
0     60876
Name: IN_TRANSPORTE_PUBLICO, dtype: int64

In [44]:
alunos_rj_16['IN_TRANSPORTE_PUBLICO'].value_counts(normalize=True)

1    0.66705
0    0.33295
Name: IN_TRANSPORTE_PUBLICO, dtype: float64

In [45]:
# Evadidos: maior percentual dentre os que não utilizam transporte público
ev = alunos_rj_16[alunos_rj_16['IN_EVASAO'] == 1]['IN_TRANSPORTE_PUBLICO'].value_counts()
ev

1    37726
0    24956
Name: IN_TRANSPORTE_PUBLICO, dtype: int64

In [46]:
ev/n

1    0.309326
0    0.409948
Name: IN_TRANSPORTE_PUBLICO, dtype: float64

* Responsável e tipos de transporte (só válidos para os 67% que usam): 99% anda de ônibus

In [47]:
alunos_rj_16[alunos_rj_16['TP_RESPONSAVEL_TRANSPORTE'].isnull() == True]['IN_TRANSPORTE_PUBLICO'].unique()

array([0])

In [48]:
alunos_rj_16[alunos_rj_16['TP_RESPONSAVEL_TRANSPORTE'].isnull() == False]['IN_TRANSPORTE_PUBLICO'].unique()

array([1])

In [49]:
n = alunos_rj_16['TP_RESPONSAVEL_TRANSPORTE'].value_counts(normalize=True) # Maioria estadual
n

1.0    0.966375
2.0    0.033625
Name: TP_RESPONSAVEL_TRANSPORTE, dtype: float64

In [50]:
alunos_rj_16['IN_TRANSP_ONIBUS'].value_counts(normalize=True)

1.0    0.990341
0.0    0.009659
Name: IN_TRANSP_ONIBUS, dtype: float64

In [51]:
alunos_rj_16['IN_TRANSP_TREM_METRO'].value_counts(normalize=True)

0.0    0.98807
1.0    0.01193
Name: IN_TRANSP_TREM_METRO, dtype: float64

#### Localização

In [52]:
len(alunos_rj_16['CO_ENTIDADE'].unique()) # Número de escolas

1097

* Tipo: menos de 4% rural

In [53]:
alunos_rj_16['TP_LOCALIZACAO'].value_counts() # Maioria urbana: 96% !!

1    176989
2      5849
Name: TP_LOCALIZACAO, dtype: int64

In [54]:
alunos_rj_16['TP_LOCALIZACAO'].value_counts(normalize=True)

1    0.96801
2    0.03199
Name: TP_LOCALIZACAO, dtype: float64

In [55]:
alunos_rj_16['TP_OUTRO_LOCAL_AULA'].value_counts(normalize=True) # 99% não estuda em outro local (hospital, casa, ...)

3    0.995767
2    0.003249
1    0.000984
Name: TP_OUTRO_LOCAL_AULA, dtype: float64

In [56]:
alunos_rj_16['TP_LOCALIZACAO_DIFERENCIADA'].value_counts(normalize=True)

0    0.999026
4    0.000974
Name: TP_LOCALIZACAO_DIFERENCIADA, dtype: float64

In [57]:
alunos_rj_16['IN_EDUCACAO_INDIGENA'].value_counts(normalize=True)

0    1.0
Name: IN_EDUCACAO_INDIGENA, dtype: float64

* Escola: 37% no Rio, resto espalhado

In [58]:
alunos_rj_16['CO_MUNICIPIO'].value_counts(normalize=True) # 37% dos alunos estudam no Rio

3304557    0.376027
3301702    0.065774
3303500    0.057417
3304904    0.046801
3305109    0.035288
3300456    0.028178
3301009    0.027347
3303302    0.027128
3302502    0.018629
3303906    0.016178
3306305    0.015484
3301900    0.014833
3303203    0.014390
3302403    0.013690
3304144    0.013449
3300100    0.011682
3305802    0.011196
3300407    0.010200
3302858    0.010096
3303401    0.009150
3302007    0.009134
3300704    0.008751
3304201    0.008439
3304524    0.007641
3302700    0.007559
3300209    0.006684
3302270    0.006158
3305554    0.005650
3302205    0.005546
3305208    0.005535
             ...   
3304409    0.001340
3302254    0.001252
3301504    0.001220
3301108    0.001187
3301207    0.001143
3303104    0.001132
3305158    0.001072
3304102    0.001034
3304110    0.000870
3303955    0.000870
3300936    0.000826
3301603    0.000826
3300258    0.000788
3302056    0.000755
3305703    0.000738
3301157    0.000678
3304128    0.000629
3300902    0.000607
3306156    0.000607


In [59]:
len(alunos_rj_16['CO_MUNICIPIO'].unique())

92

In [60]:
# Percentual de preenchimento
n = len(alunos_rj_16[alunos_rj_16['CO_MUNICIPIO'].isnull() == False])
n/len(alunos_rj_16)

1.0

In [61]:
# Região metropolitana (sem Rio)
"""
São Gonçalo, Duque de Caxias, Nova Iguaçu, Niterói, Belford Roxo, São João de Meriti, Petrópolis, Magé, 
Itaboraí, Mesquita, Nilópolis, Maricá, Queimados, Itaguaí, Japeri, Seropédica, Rio Bonito, Guapimirim, 
Cachoeiras de Macacu, Paracambi Tanguá
"""

regioes = [3304904, 3301702, 3303500, 3303302, 3300456, 3305109, 
          3303906, 3302502, 3301900, 3302858, 3303203, 3302700, 
          3304144, 3302007, 3302270, 3305554, 3304300, 3301850, 
          3300803, 3303609, 3305752]

In [62]:
# 0: Rio, 1: Grande Rio, 2: Outras regiões no estado
alunos_rj_16['IN_LOCAL_ESCOLA'] = np.where(alunos_rj_16['CO_MUNICIPIO'] == 3304557, 0, 
                                           (np.where(alunos_rj_16['CO_MUNICIPIO'].isin(regioes), 1, 2)))

In [63]:
n = alunos_rj_16['IN_LOCAL_ESCOLA'].value_counts()
n

1    71725
0    68752
2    42361
Name: IN_LOCAL_ESCOLA, dtype: int64

In [64]:
n = alunos_rj_16['IN_LOCAL_ESCOLA'].value_counts(normalize=True)
n

1    0.392287
0    0.376027
2    0.231686
Name: IN_LOCAL_ESCOLA, dtype: float64

In [65]:
# Evadidos: maior percentual de quem é do Rio, muito próximo em valores absolutos do Grande Rio
ev = alunos_rj_16[alunos_rj_16['IN_EVASAO'] == 1]['IN_LOCAL_ESCOLA'].value_counts()
ev

0    26923
1    22185
2    13574
Name: IN_LOCAL_ESCOLA, dtype: int64

In [66]:
ev/n

0    71598.607662
1    56552.959638
2    58587.923137
Name: IN_LOCAL_ESCOLA, dtype: float64

In [67]:
# alunos_rj_16['IN_ESCOLA_RIO'] = np.where(alunos_rj_16['CO_MUNICIPIO'] == 3304557, 1, 0)

# # Evadidos: maior percentual de quem não é do Rio
# alunos_rj_16[alunos_rj_16['IN_EVASAO'] == 1]['IN_ESCOLA_RIO'].value_counts(normalize=True)

# # Não evadidos
# alunos_rj_16[alunos_rj_16['IN_EVASAO'] == 0]['IN_ESCOLA_RIO'].value_counts(normalize=True)

#### Dados da turma
- 70% entre 20 e 40 alunos

In [68]:
# Percentual de preenchimento
n = len(alunos_rj_16[alunos_rj_16['ID_TURMA'].isnull() == False])
n/len(alunos_rj_16)

1.0

In [69]:
n_alunos_turma = alunos_rj_16[['ID_TURMA', 'ID']].groupby('ID_TURMA').count().rename({'ID': 'N_ALUNOS_TURMA'}, axis=1)
len(n_alunos_turma)

5851

In [70]:
alunos_rj_16['N_ALUNOS_TURMA'] = alunos_rj_16['ID_TURMA'].map(n_alunos_turma['N_ALUNOS_TURMA'])

In [71]:
len(alunos_rj_16)/5851 # Média de alunos pelo total de turmas

31.249017262006493

In [72]:
n_alunos_turma['N_ALUNOS_TURMA'].value_counts()[:10]

33    286
35    282
32    277
34    268
29    266
36    265
31    251
37    250
38    250
30    242
Name: N_ALUNOS_TURMA, dtype: int64

In [73]:
# Distribuição geral
bins = [-np.inf, 30, 40, np.inf]

bins_turma = pd.cut(n_alunos_turma['N_ALUNOS_TURMA'], bins)
bins_turma.value_counts(normalize=True)

(-inf, 30.0]    0.438728
(30.0, 40.0]    0.426936
(40.0, inf]     0.134336
Name: N_ALUNOS_TURMA, dtype: float64

In [74]:
alunos_rj_16['N_ALUNOS_TURMA_BIN'] = pd.cut(alunos_rj_16['N_ALUNOS_TURMA'], bins)
alunos_rj_16['N_ALUNOS_TURMA_BIN'].value_counts(normalize=True)

(30.0, 40.0]    0.480890
(-inf, 30.0]    0.330621
(40.0, inf]     0.188489
Name: N_ALUNOS_TURMA_BIN, dtype: float64

In [75]:
# Evadidos: 47% dos jovens evadidos são de turmas de 30-40 alunos
alunos_rj_16[alunos_rj_16['IN_EVASAO'] == 1]['N_ALUNOS_TURMA_BIN'].value_counts(normalize=True)

(30.0, 40.0]    0.474841
(-inf, 30.0]    0.324990
(40.0, inf]     0.200169
Name: N_ALUNOS_TURMA_BIN, dtype: float64

In [76]:
alunos_rj_16['N_TURMA'] = pd.cut(alunos_rj_16['N_ALUNOS_TURMA'], bins, labels=range(0,3))

In [77]:
# Checando distribuição
alunos_rj_16['N_TURMA'].value_counts(normalize=True)

1    0.480890
0    0.330621
2    0.188489
Name: N_TURMA, dtype: float64

* Dias de atividade: 98,5% 5 dias

In [78]:
alunos_rj_16['NU_DIAS_ATIVIDADE'].value_counts(normalize=True)

5    0.985309
6    0.014691
Name: NU_DIAS_ATIVIDADE, dtype: float64

In [79]:
n = alunos_rj_16['NU_DURACAO_TURMA']/60
n.value_counts(normalize=True)

5.250000     0.564385
4.500000     0.142126
5.333333     0.134234
4.750000     0.053474
5.166667     0.020067
4.666667     0.017972
5.000000     0.009889
10.000000    0.009582
4.833333     0.006968
11.833333    0.004534
11.333333    0.004255
8.000000     0.004129
9.666667     0.002653
11.000000    0.002571
9.500000     0.002543
9.583333     0.001974
9.416667     0.001892
5.500000     0.001810
9.333333     0.001783
9.250000     0.001701
9.916667     0.001695
11.166667    0.001603
9.833333     0.001493
8.833333     0.001247
4.583333     0.001094
9.166667     0.001034
11.250000    0.000624
5.666667     0.000481
4.916667     0.000459
7.166667     0.000454
9.750000     0.000284
8.666667     0.000257
4.333333     0.000224
3.833333     0.000191
3.750000     0.000153
7.000000     0.000104
10.500000    0.000060
Name: NU_DURACAO_TURMA, dtype: float64

* Tipo de turma

In [80]:
alunos_rj_16['TP_TIPO_TURMA'].value_counts(normalize=True)

0    1.0
Name: TP_TIPO_TURMA, dtype: float64

In [81]:
alunos_rj_16['TP_UNIFICADA'].value_counts(normalize=True)

0    1.0
Name: TP_UNIFICADA, dtype: float64

* Atividade complementar: 99% não tem

In [82]:
alunos_rj_16['NU_DUR_ATIV_COMP_MESMA_REDE'].value_counts(normalize=True)

0      0.999612
20     0.000252
40     0.000066
240    0.000027
96     0.000022
126    0.000016
315    0.000005
Name: NU_DUR_ATIV_COMP_MESMA_REDE, dtype: float64

In [83]:
alunos_rj_16['NU_DUR_ATIV_COMP_OUTRAS_REDES'].value_counts(normalize=True)

0      0.999557
24     0.000098
48     0.000060
30     0.000060
12     0.000038
36     0.000033
16     0.000027
240    0.000022
96     0.000022
72     0.000016
42     0.000011
280    0.000011
150    0.000011
120    0.000011
40     0.000005
38     0.000005
180    0.000005
20     0.000005
Name: NU_DUR_ATIV_COMP_OUTRAS_REDES, dtype: float64

In [84]:
alunos_rj_16['NU_DUR_AEE_MESMA_REDE'].value_counts(normalize=True)

0      0.999650
120    0.000060
240    0.000044
60     0.000038
84     0.000033
36     0.000022
96     0.000022
180    0.000022
48     0.000016
192    0.000016
207    0.000016
189    0.000011
320    0.000011
315    0.000011
72     0.000011
232    0.000005
144    0.000005
126    0.000005
Name: NU_DUR_AEE_MESMA_REDE, dtype: float64

In [85]:
alunos_rj_16['NU_DUR_AEE_OUTRAS_REDES'].value_counts(normalize=True)

0      0.999874
240    0.000049
144    0.000038
60     0.000011
54     0.000011
108    0.000005
192    0.000005
30     0.000005
Name: NU_DUR_AEE_OUTRAS_REDES, dtype: float64

In [86]:
alunos_rj_16['TP_MEDIACAO_DIDATICO_PEDAGO'].value_counts(normalize=True)

1    1.0
Name: TP_MEDIACAO_DIDATICO_PEDAGO, dtype: float64

### Filtragem das variáveis

In [87]:
out_outro_campo = ['NU_ANO_CENSO', 'ID_MATRICULA', 'NU_ANO', 'NU_DIA', 'NU_MES', 'NU_DIA',  'NU_IDADE', 
                   'CO_REGIAO','CO_MESORREGIAO', 'CO_MICRORREGIAO', 'CO_UF', 'ID_TURMA',
                   'IN_REGULAR', 'IN_EJA', 'TP_REGULAMENTACAO', 'CO_DISTRITO', 'IN_EDUCACAO_INDIGENA'] # Já temos info
len(out_outro_campo)

17

In [88]:
out_variacao = ['NU_DURACAO_TURMA', 'NU_DUR_ATIV_COMP_MESMA_REDE', 'NU_DUR_ATIV_COMP_OUTRAS_REDES',
                'NU_DUR_AEE_MESMA_REDE', 'NU_DUR_AEE_OUTRAS_REDES', 'NU_DIAS_ATIVIDADE', 'TP_NACIONALIDADE', 
                'CO_PAIS_ORIGEM', 'CO_UF_NASC', 'TP_OUTRO_LOCAL_AULA', 'TP_RESPONSAVEL_TRANSPORTE', 
                'IN_TRANSP_VANS_KOMBI', 'IN_TRANSP_MICRO_ONIBUS', 'IN_TRANSP_ONIBUS', 'IN_TRANSP_BICICLETA',
                'IN_TRANSP_TR_ANIMAL', 'IN_TRANSP_OUTRO_VEICULO', 'IN_TRANSP_EMBAR_ATE5', 'IN_TRANSP_EMBAR_5A15', 
                'IN_TRANSP_EMBAR_15A35', 'IN_TRANSP_EMBAR_35', 'IN_TRANSP_TREM_METRO', 'IN_NECESSIDADE_ESPECIAL',
                'TP_MEDIACAO_DIDATICO_PEDAGO', 'IN_ESPECIAL_EXCLUSIVA', 'IN_PROFISSIONALIZANTE', 
                'TP_DEPENDENCIA', 'TP_LOCALIZACAO', 'TP_LOCALIZACAO_DIFERENCIADA', 'TP_UNIFICADA', 'TP_TIPO_TURMA'] # Baixa variacao

len(out_variacao)

31

In [89]:
alunos_rj_16.columns

Index(['Unnamed: 0', 'NU_ANO_CENSO', 'ID_MATRICULA', 'NU_DIA', 'NU_MES',
       'NU_ANO', 'NU_IDADE_REFERENCIA', 'NU_IDADE', 'NU_DURACAO_TURMA',
       'NU_DUR_ATIV_COMP_MESMA_REDE', 'NU_DUR_ATIV_COMP_OUTRAS_REDES',
       'NU_DUR_AEE_MESMA_REDE', 'NU_DUR_AEE_OUTRAS_REDES', 'NU_DIAS_ATIVIDADE',
       'TP_SEXO', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM',
       'CO_UF_NASC', 'CO_MUNICIPIO_NASC', 'TP_ZONA_RESIDENCIAL',
       'TP_OUTRO_LOCAL_AULA', 'IN_TRANSPORTE_PUBLICO',
       'TP_RESPONSAVEL_TRANSPORTE', 'IN_TRANSP_VANS_KOMBI',
       'IN_TRANSP_MICRO_ONIBUS', 'IN_TRANSP_ONIBUS', 'IN_TRANSP_BICICLETA',
       'IN_TRANSP_TR_ANIMAL', 'IN_TRANSP_OUTRO_VEICULO',
       'IN_TRANSP_EMBAR_ATE5', 'IN_TRANSP_EMBAR_5A15', 'IN_TRANSP_EMBAR_15A35',
       'IN_TRANSP_EMBAR_35', 'IN_TRANSP_TREM_METRO', 'IN_NECESSIDADE_ESPECIAL',
       'TP_MEDIACAO_DIDATICO_PEDAGO', 'IN_ESPECIAL_EXCLUSIVA', 'IN_REGULAR',
       'IN_EJA', 'IN_PROFISSIONALIZANTE', 'TP_ETAPA_ENSINO', 'ID_TURMA',
       'TP

In [90]:
len(alunos_rj_16.columns)

64

In [91]:
in_cols = [i for i in alunos_rj_16.columns if not i in out_variacao]
len(in_cols)

33

In [92]:
in_cols = [i for i in in_cols if not i in out_outro_campo]
len(in_cols)

17

In [93]:
in_cols

['Unnamed: 0',
 'NU_IDADE_REFERENCIA',
 'TP_SEXO',
 'TP_COR_RACA',
 'CO_MUNICIPIO_NASC',
 'TP_ZONA_RESIDENCIAL',
 'IN_TRANSPORTE_PUBLICO',
 'TP_ETAPA_ENSINO',
 'CO_ENTIDADE',
 'CO_MUNICIPIO',
 'ID',
 'IN_EVASAO',
 'IN_DISTORCAO',
 'IN_LOCAL_ESCOLA',
 'N_ALUNOS_TURMA',
 'N_ALUNOS_TURMA_BIN',
 'N_TURMA']

In [94]:
cols = ['ID', 'CO_ENTIDADE', 'TP_SEXO', 'NU_IDADE', 'IN_DISTORCAO',
        'IN_TRANSPORTE_PUBLICO', 'IN_LOCAL_ESCOLA', 'N_TURMA', 'IN_EVASAO']

In [95]:
df_rj_16 = alunos_rj_16[cols]

## Infos das escolas: Censo 2017

In [96]:
# Total de escolas em 2016
len(df_rj_16['CO_ENTIDADE'].unique())

1097

In [97]:
escolas_rj_16 = pd.read_csv(TREAT_PATH / 'censo' / 'cleaned_escolas_rj_2016.csv')
escolas_rj_16 = escolas_rj_16.drop('Unnamed: 0', axis=1)
escolas_rj_16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1689 entries, 0 to 1688
Data columns (total 18 columns):
NU_ANO_CENSO                  1689 non-null int64
CO_ENTIDADE                   1689 non-null int64
CO_MUNICIPIO                  1689 non-null int64
IN_AGUA_FILTRADA              1324 non-null float64
IN_AGUA_INEXISTENTE           1324 non-null float64
IN_ENERGIA_INEXISTENTE        1324 non-null float64
IN_LIXO_COLETA_PERIODICA      1324 non-null float64
IN_LABORATORIO_INFORMATICA    1324 non-null float64
IN_LABORATORIO_CIENCIAS       1324 non-null float64
IN_QUADRA_ESPORTES            1324 non-null float64
IN_COZINHA                    1324 non-null float64
IN_REFEITORIO                 1324 non-null float64
IN_BIBLIOTECA                 1324 non-null float64
IN_BANHEIRO_FORA_PREDIO       1324 non-null float64
IN_BANHEIRO_DENTRO_PREDIO     1324 non-null float64
IN_AREA_VERDE                 1324 non-null float64
NU_COMP_ALUNO                 1324 non-null float64
IN_ALIMENTACAO 

In [98]:
# Todos os alunos estão em escolas que aparecem na base
alunos_rj_16[alunos_rj_16['CO_ENTIDADE'].isin(escolas_rj_16['CO_ENTIDADE']) == False]

,Unnamed: 0,NU_ANO_CENSO,ID_MATRICULA,NU_DIA,NU_MES,NU_ANO,NU_IDADE_REFERENCIA,NU_IDADE,NU_DURACAO_TURMA,NU_DUR_ATIV_COMP_MESMA_REDE,NU_DUR_ATIV_COMP_OUTRAS_REDES,NU_DUR_AEE_MESMA_REDE,NU_DUR_AEE_OUTRAS_REDES,NU_DIAS_ATIVIDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,CO_PAIS_ORIGEM,CO_UF_NASC,CO_MUNICIPIO_NASC,TP_ZONA_RESIDENCIAL,TP_OUTRO_LOCAL_AULA,IN_TRANSPORTE_PUBLICO,TP_RESPONSAVEL_TRANSPORTE,IN_TRANSP_VANS_KOMBI,IN_TRANSP_MICRO_ONIBUS,IN_TRANSP_ONIBUS,IN_TRANSP_BICICLETA,IN_TRANSP_TR_ANIMAL,IN_TRANSP_OUTRO_VEICULO,IN_TRANSP_EMBAR_ATE5,IN_TRANSP_EMBAR_5A15,IN_TRANSP_EMBAR_15A35,IN_TRANSP_EMBAR_35,IN_TRANSP_TREM_METRO,IN_NECESSIDADE_ESPECIAL,TP_MEDIACAO_DIDATICO_PEDAGO,IN_ESPECIAL_EXCLUSIVA,IN_REGULAR,IN_EJA,IN_PROFISSIONALIZANTE,TP_ETAPA_ENSINO,ID_TURMA,TP_UNIFICADA,TP_TIPO_TURMA,CO_ENTIDADE,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_REGULAMENTACAO,TP_LOCALIZACAO_DIFERENCIADA,IN_EDUCACAO_INDIGENA,ID,IN_EVASAO,IN_DISTORCAO,IN_LOCAL_ESCOLA,N_ALUNOS_TURMA,N_ALUNOS_TURMA_BIN,N_TURMA


In [99]:
df_rj_16 = df_rj_16.merge(escolas_rj_16, on='CO_ENTIDADE', how='left')
df_rj_16.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182838 entries, 0 to 182837
Data columns (total 26 columns):
ID                            182838 non-null int64
CO_ENTIDADE                   182838 non-null int64
TP_SEXO                       182838 non-null int64
NU_IDADE                      182838 non-null int64
IN_DISTORCAO                  182838 non-null int64
IN_TRANSPORTE_PUBLICO         182838 non-null int64
IN_LOCAL_ESCOLA               182838 non-null int64
N_TURMA                       182838 non-null category
IN_EVASAO                     182838 non-null int64
NU_ANO_CENSO                  182838 non-null int64
CO_MUNICIPIO                  182838 non-null int64
IN_AGUA_FILTRADA              182838 non-null float64
IN_AGUA_INEXISTENTE           182838 non-null float64
IN_ENERGIA_INEXISTENTE        182838 non-null float64
IN_LIXO_COLETA_PERIODICA      182838 non-null float64
IN_LABORATORIO_INFORMATICA    182838 non-null float64
IN_LABORATORIO_CIENCIAS       182838 non-nul

### Checando distribuições

In [100]:
cols = ['NU_ANO_CENSO', 'CO_ENTIDADE', 'CO_MUNICIPIO', 'IN_AGUA_FILTRADA', 
        'IN_AGUA_INEXISTENTE', 'IN_ENERGIA_INEXISTENTE', 'IN_LIXO_COLETA_PERIODICA', 'IN_LABORATORIO_INFORMATICA', 
        'IN_LABORATORIO_CIENCIAS', 'IN_QUADRA_ESPORTES', 'IN_COZINHA', 'IN_REFEITORIO', 'IN_BIBLIOTECA',
        'IN_BANHEIRO_FORA_PREDIO', 'IN_BANHEIRO_DENTRO_PREDIO', 'IN_AREA_VERDE', 'NU_COMP_ALUNO', 'IN_ALIMENTACAO']

check_escolas_16 = df_rj_16[cols].drop_duplicates()
check_escolas_16.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1097 entries, 0 to 34258
Data columns (total 18 columns):
NU_ANO_CENSO                  1097 non-null int64
CO_ENTIDADE                   1097 non-null int64
CO_MUNICIPIO                  1097 non-null int64
IN_AGUA_FILTRADA              1097 non-null float64
IN_AGUA_INEXISTENTE           1097 non-null float64
IN_ENERGIA_INEXISTENTE        1097 non-null float64
IN_LIXO_COLETA_PERIODICA      1097 non-null float64
IN_LABORATORIO_INFORMATICA    1097 non-null float64
IN_LABORATORIO_CIENCIAS       1097 non-null float64
IN_QUADRA_ESPORTES            1097 non-null float64
IN_COZINHA                    1097 non-null float64
IN_REFEITORIO                 1097 non-null float64
IN_BIBLIOTECA                 1097 non-null float64
IN_BANHEIRO_FORA_PREDIO       1097 non-null float64
IN_BANHEIRO_DENTRO_PREDIO     1097 non-null float64
IN_AREA_VERDE                 1097 non-null float64
NU_COMP_ALUNO                 1097 non-null float64
IN_ALIMENTACAO

In [101]:
check_escolas_16['IN_AGUA_INEXISTENTE'].value_counts()

0.0    1092
1.0       5
Name: IN_AGUA_INEXISTENTE, dtype: int64

In [102]:
df_rj_16 = df_rj_16.drop('IN_AGUA_INEXISTENTE', axis=1)

In [103]:
check_escolas_16['IN_AGUA_FILTRADA'].value_counts()

1.0    1097
Name: IN_AGUA_FILTRADA, dtype: int64

In [104]:
df_rj_16 = df_rj_16.drop('IN_AGUA_FILTRADA', axis=1)

In [105]:
check_escolas_16['IN_ENERGIA_INEXISTENTE'].value_counts()

0.0    1097
Name: IN_ENERGIA_INEXISTENTE, dtype: int64

In [106]:
df_rj_16 = df_rj_16.drop('IN_ENERGIA_INEXISTENTE', axis=1)

In [107]:
check_escolas_16['IN_LIXO_COLETA_PERIODICA'].value_counts()

1.0    1093
0.0       4
Name: IN_LIXO_COLETA_PERIODICA, dtype: int64

In [108]:
df_rj_16 = df_rj_16.drop('IN_LIXO_COLETA_PERIODICA', axis=1)

In [109]:
check_escolas_16['IN_LABORATORIO_INFORMATICA'].value_counts(normalize=True)

1.0    0.765725
0.0    0.234275
Name: IN_LABORATORIO_INFORMATICA, dtype: float64

In [110]:
check_escolas_16['IN_LABORATORIO_CIENCIAS'].value_counts(normalize=True)

1.0    0.71103
0.0    0.28897
Name: IN_LABORATORIO_CIENCIAS, dtype: float64

In [111]:
check_escolas_16['IN_QUADRA_ESPORTES'].value_counts(normalize=True)

1.0    0.915223
0.0    0.084777
Name: IN_QUADRA_ESPORTES, dtype: float64

In [112]:
check_escolas_16['IN_COZINHA'].value_counts(normalize=True)

1.0    0.990884
0.0    0.009116
Name: IN_COZINHA, dtype: float64

In [113]:
df_rj_16 = df_rj_16.drop('IN_COZINHA', axis=1)

In [114]:
check_escolas_16['IN_REFEITORIO'].value_counts(normalize=True)

1.0    0.968095
0.0    0.031905
Name: IN_REFEITORIO, dtype: float64

In [115]:
df_rj_16 = df_rj_16.drop('IN_REFEITORIO', axis=1)

In [116]:
check_escolas_16['IN_BIBLIOTECA'].value_counts(normalize=True)

1.0    0.880583
0.0    0.119417
Name: IN_BIBLIOTECA, dtype: float64

In [117]:
check_escolas_16['IN_BANHEIRO_FORA_PREDIO'].value_counts(normalize=True)

1.0    0.749316
0.0    0.250684
Name: IN_BANHEIRO_FORA_PREDIO, dtype: float64

In [118]:
check_escolas_16['IN_BANHEIRO_DENTRO_PREDIO'].value_counts(normalize=True)

1.0    0.990884
0.0    0.009116
Name: IN_BANHEIRO_DENTRO_PREDIO, dtype: float64

In [119]:
df_rj_16 = df_rj_16.drop('IN_BANHEIRO_DENTRO_PREDIO', axis=1)

In [120]:
check_escolas_16['IN_AREA_VERDE'].value_counts(normalize=True)

1.0    0.859617
0.0    0.140383
Name: IN_AREA_VERDE, dtype: float64

In [121]:
check_escolas_16['IN_ALIMENTACAO'].value_counts(normalize=True)

1.0    1.0
Name: IN_ALIMENTACAO, dtype: float64

In [122]:
df_rj_16 = df_rj_16.drop('IN_ALIMENTACAO', axis=1)

In [123]:
df_rj_16['NU_COMP_ALUNO'].value_counts()

0.0      33627
10.0     32613
20.0      9566
9.0       8680
17.0      7388
19.0      6961
8.0       6913
15.0      6822
6.0       6747
12.0      6502
7.0       6409
5.0       6239
13.0      5360
18.0      5024
14.0      4510
16.0      4206
22.0      3433
11.0      2651
25.0      2626
23.0      1792
24.0      1474
21.0      1464
27.0      1387
28.0      1190
26.0      1057
60.0       827
1.0        732
36.0       668
30.0       655
70.0       606
4.0        572
3.0        507
29.0       504
68.0       360
50.0       351
55.0       343
32.0       316
76.0       293
31.0       249
34.0       231
281.0      219
40.0       179
72.0       166
138.0      114
67.0        88
38.0        83
49.0        72
115.0       62
Name: NU_COMP_ALUNO, dtype: int64

In [124]:
# Número médio por turma: 30 / 2 alunos por computador na aula = 15
pd.cut(df_rj_16['NU_COMP_ALUNO'], [-np.inf, 15, np.inf]).value_counts(normalize=True)

(-inf, 15.0]    0.704908
(15.0, inf]     0.295092
Name: NU_COMP_ALUNO, dtype: float64

In [125]:
df_rj_16['IN_N_COMP_15'] = pd.cut(df_rj_16['NU_COMP_ALUNO'], [-np.inf, 15, np.inf], labels=[0,1])
df_rj_16['IN_N_COMP_15'].value_counts(normalize=True)

0    0.704908
1    0.295092
Name: IN_N_COMP_15, dtype: float64

In [126]:
df_rj_16.columns

Index(['ID', 'CO_ENTIDADE', 'TP_SEXO', 'NU_IDADE', 'IN_DISTORCAO',
       'IN_TRANSPORTE_PUBLICO', 'IN_LOCAL_ESCOLA', 'N_TURMA', 'IN_EVASAO',
       'NU_ANO_CENSO', 'CO_MUNICIPIO', 'IN_LABORATORIO_INFORMATICA',
       'IN_LABORATORIO_CIENCIAS', 'IN_QUADRA_ESPORTES', 'IN_BIBLIOTECA',
       'IN_BANHEIRO_FORA_PREDIO', 'IN_AREA_VERDE', 'NU_COMP_ALUNO',
       'IN_N_COMP_15'],
      dtype='object')

## Nível socioeconômico

- Somente 936 escolas estaduais no RJ possuem o indicador de nível socioeconômico
- Dentre nosso grupo de interesse temos info de apenas 849 das 1097 escolas: as faltantes concentram 20% dos alunos evadidos!

In [127]:
inse_rj_2015 = pd.read_csv(TREAT_PATH / 'inse_2015_rj.csv')
inse_rj_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 936 entries, 0 to 935
Data columns (total 14 columns):
Unnamed: 0             936 non-null int64
CO_ESCOLA              936 non-null int64
NOME_ESCOLA            936 non-null object
CO_UF                  936 non-null int64
NOME_UF                936 non-null object
CO_MUNICIPIO           936 non-null int64
NOME_MUNICIPIO         936 non-null int64
ID_AREA                936 non-null int64
TP_DEPENDENCIA         936 non-null int64
TP_LOCALIZACAO         936 non-null int64
QTD_ALUNOS_INSE        936 non-null int64
INSE_VALOR_ABSOLUTO    936 non-null float64
INSE_CLASSIFICACAO     936 non-null object
NIVEL                  936 non-null int64
dtypes: float64(1), int64(10), object(3)
memory usage: 102.5+ KB


In [128]:
len(df_rj_16[df_rj_16['CO_ENTIDADE'].isin(inse_rj_2015['CO_ESCOLA'])]['CO_ENTIDADE'].drop_duplicates())

849

In [129]:
len(df_rj_16[df_rj_16['CO_ENTIDADE'].isin(inse_rj_2015['CO_ESCOLA'])])/len(df_rj_16)

0.8162088843675822

In [130]:
inse_rj_2015 = inse_rj_2015.rename({'CO_ESCOLA': 'CO_ENTIDADE'}, axis=1)

df_rj_16 = df_rj_16.merge(inse_rj_2015[['CO_ENTIDADE', 'NIVEL']], on='CO_ENTIDADE', how='left')
df_rj_16.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182838 entries, 0 to 182837
Data columns (total 20 columns):
ID                            182838 non-null int64
CO_ENTIDADE                   182838 non-null int64
TP_SEXO                       182838 non-null int64
NU_IDADE                      182838 non-null int64
IN_DISTORCAO                  182838 non-null int64
IN_TRANSPORTE_PUBLICO         182838 non-null int64
IN_LOCAL_ESCOLA               182838 non-null int64
N_TURMA                       182838 non-null category
IN_EVASAO                     182838 non-null int64
NU_ANO_CENSO                  182838 non-null int64
CO_MUNICIPIO                  182838 non-null int64
IN_LABORATORIO_INFORMATICA    182838 non-null float64
IN_LABORATORIO_CIENCIAS       182838 non-null float64
IN_QUADRA_ESPORTES            182838 non-null float64
IN_BIBLIOTECA                 182838 non-null float64
IN_BANHEIRO_FORA_PREDIO       182838 non-null float64
IN_AREA_VERDE                 182838 non-nul

In [131]:
df_rj_16[df_rj_16['IN_EVASAO'] == 1]['NIVEL'].fillna(-1).value_counts()

 3.0    42760
-1.0    14433
 4.0     4950
 2.0      359
 5.0      158
 1.0       13
 6.0        9
Name: NIVEL, dtype: int64

In [132]:
14670/len(df_rj_16[df_rj_16['IN_EVASAO'] == 1])

0.2340384799463961

### Finalizando a base

In [133]:
# df_work = df_rj_16[df_rj_16['NIVEL'].isnull() == False]
df_work = df_rj_16.drop(['NU_COMP_ALUNO'], axis=1)
df_work.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182838 entries, 0 to 182837
Data columns (total 19 columns):
ID                            182838 non-null int64
CO_ENTIDADE                   182838 non-null int64
TP_SEXO                       182838 non-null int64
NU_IDADE                      182838 non-null int64
IN_DISTORCAO                  182838 non-null int64
IN_TRANSPORTE_PUBLICO         182838 non-null int64
IN_LOCAL_ESCOLA               182838 non-null int64
N_TURMA                       182838 non-null category
IN_EVASAO                     182838 non-null int64
NU_ANO_CENSO                  182838 non-null int64
CO_MUNICIPIO                  182838 non-null int64
IN_LABORATORIO_INFORMATICA    182838 non-null float64
IN_LABORATORIO_CIENCIAS       182838 non-null float64
IN_QUADRA_ESPORTES            182838 non-null float64
IN_BIBLIOTECA                 182838 non-null float64
IN_BANHEIRO_FORA_PREDIO       182838 non-null float64
IN_AREA_VERDE                 182838 non-nul

In [134]:
df_work['IN_EVASAO'].value_counts(normalize=True)

0    0.657172
1    0.342828
Name: IN_EVASAO, dtype: float64

In [135]:
df_work[df_work['NIVEL'].isnull() == False]['IN_EVASAO'].value_counts(normalize=True)

0    0.676689
1    0.323311
Name: IN_EVASAO, dtype: float64

In [136]:
df_work.to_csv(TREAT_PATH / 'analysis' / 'df_alunos.csv')